# Test data generator
## In this notebook I will generate test data with the method described in [Drexl's paper](files/Drexl_branchAndCut_VRPTT.pdf)

check whether directory present to save test data

In [ ]:
DATA_DIR = "test_data"
if not os.path.isdir('.'+DATA_DIR):
    !mkdir DATA_DIR

### Instance description 
The customer and transshipment locations are randomly
selected on a 100 × 100 km grid with the depot located in
the center. The resulting Euclidean distances between each
pair of vertices are multiplied by a distance factor of 1.3. The
customer supplies are chosen randomly from[1,000; 10,000].
As load transfer time, 2 min per 1,000 units of supply are
assumed.
The length of the planning horizon is 22 h (1,320 min). All
customer and transshipment locations have a time window
of [0; 1,320]. Such nonrestrictive time windows represent
the situation in raw milk collection where only very few
customers or TLs actually have time windows.
The test instances were created with enough vehicles of
each class such that the complete supply could be collected
with either only lorries of class 1 and trailers of class 2 or
only lorries of class 2 and trailers of class 1.
With these parameters, a set of so-called x_y_z instances
was created, where 2 ≤ x ≤ 20 stands for the number of
customers, 2 ≤ y ≤ 20 for the number of potential TLs,
and 4 ≤ z ≤ 24 for the number of vehicles. It is always
x = y, and there are always x/2 lorry customers and x/2
trailer customers. y/2 potential TLs correspond to the trailer
customer locations, and y/2 are pure TLs. For each of the
four vehicle classes, there are z/4 vehicles. This means that
an x_y_z instance has 1+x +3 ·

k
∈FT |Fk
|· y+1 vertices:
one for the start depot, x for the customers, three for each
trailer at each of the y potential TLs to represent decoupling,
transfer, and recoupling, and one for the end depot. Table 2
shows details on instance sizes.

In [39]:
import tables as tb
tb.open_file?

In [40]:
f = tb.open_file('temp.h5', 'w')

In [43]:
import blaze


In [45]:
help(blaze)

Help on package blaze:

NAME
    blaze

FILE
    /home/sandralaptop/anaconda2/lib/python2.7/site-packages/blaze/__init__.py

PACKAGE CONTENTS
    _version
    cached
    compatibility
    compute (package)
    dispatch
    expr (package)
    index
    interactive
    mongo
    partition
    server (package)
    sql
    tests (package)
    utils

SUBMODULES
    datetime

DATA
    Broadcastable = (<class 'blaze.expr.arithmetic.BinOp'>, <class 'blaze....
    Cheap = (<class 'blaze.expr.collections.Head'>, <class 'blaze.expr.exp...
    Sequence = (<type 'tuple'>, <type 'list'>, <class '_abcoll.Iterator'>,...
    __version__ = u'0.9.0'
    abs = <dispatched abs>
    absolute_import = _Feature((2, 5, 0, 'alpha', 1), (3, 0, 0, 'alpha', 0...
    acos = <dispatched acos>
    acosh = <dispatched acosh>
    all = <dispatched all>
    all_formats = frozenset([SerializationFormat(name='json', loads=<funct...
    any = <dispatched any>
    api = <flask.blueprints.Blueprint object>
    append = <disp

In [47]:
import os.path
import shutil
from time import time
import blaze
import tables

N = 500

if os.path.exists('c'):
    shutil.rmtree('c')

a = [ u"s"*N*i for i in range(N) ]

t0 = time()
c = blaze.Data(a)#, 'x, object', params=blaze.params(storage='c', clevel=5))
print "time taken for writing in Blaze: %.3f" % (time() - t0)

t0 = time()
#c2 = blaze.open('c')
c2 = c
#print c2.datashape

tlen = 0
for i in range(N):
    #print "i:", i, repr(c2[i]), type(c2[i])
    tlen += len(c2[i][()])
print "time taken for reading in Blaze: %.3f" % (time() - t0)
print "tlen", tlen


# Create a VLArray:
t0 = time()
f = tables.openFile('vlarray.h5', mode='w')
vlarray = f.createVLArray(f.root, 'vlarray',
                          tables.ObjectAtom(),
                          "array of objects",
                          filters=tables.Filters(5))

for obj in a:
    vlarray.append(obj)
f.close()
print "time taken for writing in HDF5: %.3f" % (time() - t0)

# Read the VLArray:
t0 = time()
f = tables.openFile('vlarray.h5', mode='r')
vlarray = f.root.vlarray

tlen = 0
for obj in vlarray:
    tlen += len(obj)
f.close()
print "time taken for reading in HDF5: %.3f" % (time() - t0)
print "tlen", tlen

time taken for writing in Blaze: 5.310


RuntimeError: maximum recursion depth exceeded